# Evaluación Módulo 3 : Programa lealtad de una aerolínea

## 1.- EDA:Exploración inicial de los datos

In [ ]:
# En primer lugar, importamos las librerías que nos harán falta:

# Para tratamiento de datos

import pandas as pd 
import numpy as np 

# Procesos de imputación

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Visuaicación de resultados

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None) 

In [20]:
# Carga de csv (1):

df_flight = pd.read_csv("files/Customer_Flight_Activity.csv")

df_flight.head(2)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0


In [60]:
# Carga de csv (2) :

df_loyalty = pd.read_csv("files/Customer_Loyalty_History.csv")

df_loyalty.head(2)


,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN


### EDA: Customer_Flight_Activity.csv

In [ ]:
# Forma del conjunto de daro: (filas, columnas)

df_flight.shape

(405624, 10)

In [ ]:
# Porcentaje de nulos por columna

df_flight.isna().sum()/df_flight.shape[0]*100

Loyalty Number                 0.0
Year                           0.0
Month                          0.0
Flights Booked                 0.0
Flights with Companions        0.0
Total Flights                  0.0
Distance                       0.0
Points Accumulated             0.0
Points Redeemed                0.0
Dollar Cost Points Redeemed    0.0
dtype: float64

In [ ]:
# Duplicados totales del conjunto de datos

df_flight.duplicated().sum()

np.int64(1864)

In [ ]:
"""Muestra de duplicados del conjunto de datos, de la columna "Loyalty Number", que es la columna
que podría causar conflictos, ya que representa un identificador único"""

# Seguimos con el EDA, también del archivo: Customer_Loyalty_History.csv antes de tomar decisiones

df_flight[df_flight.duplicated(subset="Loyalty Number", keep=False)].sort_values("Loyalty Number").head(3)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
169010,100018,2017,11,13,1,14,3374,337.0,0,0
135208,100018,2017,9,9,4,13,1651,165.0,0,0
50703,100018,2017,4,4,0,4,924,92.0,0,0


In [29]:
# Revisión de columnas

def revision_columnas(df):

    for c in df.columns:
        
        print(f"Valores únicos en la columna '{c}':\n{df[c].unique()}\n")


In [ ]:
"""Aunque no todas las columnas requieren un análisis de valores únicos,
se aplica unique() de forma general mediante una función para mantener un
proceso uniforme y aprovechar la eficiencia del procesamiento en bloque."""

revision_columnas(df_flight)

Valores únicos en la columna 'Loyalty Number':
[100018 100102 100140 ... 999731 999788 999891]

Valores únicos en la columna 'Year':
[2017 2018]

Valores únicos en la columna 'Month':
[ 1  9  2  3 11  4  5  7  6  8 10 12]

Valores únicos en la columna 'Flights Booked':
[ 3 10  6  0  8 11  9  4  7  5  2  1 12 13 14 16 15 17 18 19 20 21]

Valores únicos en la columna 'Flights with Companions':
[ 0  4  7  1  6  3  5  2 10  8  9 11]

Valores únicos en la columna 'Total Flights':
[ 3 14  6  0 15 11 12 10  8  9  7  5 16  2  1 17 13 22  4 19 18 21 26 20
 23 25 27 24 28 30 29 31 32]

Valores únicos en la columna 'Distance':
[1521 2030 1200 ... 1217  617 4135]

Valores únicos en la columna 'Points Accumulated':
[152.   203.   120.   ...  18.75 601.   626.  ]

Valores únicos en la columna 'Points Redeemed':
[  0 341 364 310 445 312 343 366 389 292 447 324 456 409 436 327 322 291
 323 300 290 309 325 386 321 363 340 670 443 517 444 328 344 367 313 333
 293 449 297 455 372 356 405 381 466 419 369 

In [ ]:
df_flight.info()

# Los datos son de tipo "int" excepto puntos acumulados, que son de tipo float.
# La documentación no especifíca, que los puntos puedan ser con decimales

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


In [ ]:
# Podemos hacer una búsqueda de números decimales, o si son todos enteros, convertir a int (no hay nulos).

# OPCIONES: Con esta función, o haciendo un .sample() y a ojo.

def tiene_decimales_reales(columna):
    return ((columna %1)!= 0).sum()

#  columna %1 --> obtiene el residuo de dividir cada valor de la columna por 1
# y lo compara con que sea diferente a 0 con !=0
# sum() nos saca cuantos hay


In [ ]:
tiene_decimales_reales(df_flight['Points Accumulated'])

# Sí, esto quiere decir que hay puntos acumulados fraccionados. Tipo: "23.5"

np.int64(20937)

### EDA: Customer_Loyalty_History.csv

In [ ]:
### EDA: Customer_Flight_Activity.csv